In [5]:
# base_path = './'

In [6]:
import pandas as pd
from time import time
import tensorflow as tf
import os

In [7]:
def gerar_rede(num_classes: int, n_neurons: int, n_hidden_layers: int):
    dir_path = os.getcwd()
    
    metrics_file = f"{dir_path}/training_metrics.csv"
    if os.path.exists(metrics_file):
        metrics_df = pd.read_csv(metrics_file)
    else:
        metrics_df = pd.DataFrame(columns=["Model Name", "Training Time", "Train Accuracy", "Test Accuracy"])

    file_data_train = os.path.join(dir_path, "wine_train.csv")
    file_data_test = os.path.join(dir_path, "wine_test.csv")
    
    data_train = pd.read_csv(file_data_train).to_numpy()
    data_test = pd.read_csv(file_data_test).to_numpy()

    x_train, y_train = data_train[:, :-1], data_train[:, -1]
    x_test, y_test = data_test[:, :-1], data_test[:, -1]

    y_train_ohe = tf.keras.utils.to_categorical(
        y_train, num_classes=num_classes)
    y_test_ohe = tf.keras.utils.to_categorical(y_test, num_classes=num_classes)

    model = tf.keras.Sequential(
        [
            tf.keras.layers.Input(shape=[x_train.shape[1]]),
        ]
    )

    for _ in range(n_hidden_layers):
        model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))

    model.add(tf.keras.layers.Dense(num_classes, activation="softmax"))

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    
    extentions = ['h5', 'keras']
    model_path = os.path.join(
        dir_path, "models", f"model_{n_hidden_layers}layers_{n_neurons}neurons"
    )
    if not os.path.join(dir_path, "models"):
        os.makedirs(os.path.join(dir_path, "models"))
    

    es = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
    ck = tf.keras.callbacks.ModelCheckpoint(
        f'{model_path}.{extentions[1]}', monitor="val_accuracy", save_best_only=True
    )

    start = time()
    model.fit(
        x_train,
        y_train_ohe,
        batch_size=4,
        epochs=100,
        validation_data=(x_test, y_test_ohe),
        verbose=2,
        callbacks=[ck, es],
    )
    end = time() 
    
    print(f"Tempo de Treinamento: {end-start}")

    # salvar modelo
    model = tf.keras.models.load_model(f'{model_path}.{extentions[1]}')

    # avaliar modelo com os dados de treinamento
    print("Resultado Treinamento")
    model.evaluate(x_train, y_train_ohe, verbose=2)

    # avaliar modelo com os dados de teste
    print("Resultado Teste")
    # model.evaluate(x_test, y_test_ohe, verbose=2)
    model.save(f'{model_path}.{extentions[0]}')
    
    train_loss, train_acc = model.evaluate(x_train, y_train_ohe, verbose=2)
    test_loss, test_acc = model.evaluate(x_test, y_test_ohe, verbose=2)
    
    metrics_df.loc[len(metrics_df)] = { # type: ignore
        "Model Name": f"model_{n_hidden_layers}layers_{n_neurons}neurons",
        "Training Time": end - start,
        "Train Accuracy": train_acc,
        "Test Accuracy": test_acc
    } 
    metrics_df.to_csv(metrics_file, index=False)

In [11]:
n_hidden_layers = 1
n_neurons = 20
num_classes = 3
gerar_rede(num_classes, n_neurons, n_hidden_layers)

Epoch 1/100
36/36 - 2s - 57ms/step - accuracy: 0.4507 - loss: 1.0320 - val_accuracy: 0.4722 - val_loss: 0.9869
Epoch 2/100
36/36 - 0s - 4ms/step - accuracy: 0.5141 - loss: 0.9959 - val_accuracy: 0.6111 - val_loss: 0.9371
Epoch 3/100
36/36 - 0s - 6ms/step - accuracy: 0.5704 - loss: 0.9641 - val_accuracy: 0.6944 - val_loss: 0.8945
Epoch 4/100
36/36 - 1s - 21ms/step - accuracy: 0.5845 - loss: 0.9412 - val_accuracy: 0.6667 - val_loss: 0.8585
Epoch 5/100
36/36 - 0s - 4ms/step - accuracy: 0.6056 - loss: 0.9176 - val_accuracy: 0.6389 - val_loss: 0.8330
Epoch 6/100
36/36 - 0s - 4ms/step - accuracy: 0.5775 - loss: 0.8979 - val_accuracy: 0.6667 - val_loss: 0.8070
Epoch 7/100
36/36 - 0s - 4ms/step - accuracy: 0.6197 - loss: 0.8809 - val_accuracy: 0.6944 - val_loss: 0.7842
Epoch 8/100
36/36 - 0s - 7ms/step - accuracy: 0.6338 - loss: 0.8634 - val_accuracy: 0.7222 - val_loss: 0.7630
Epoch 9/100
36/36 - 0s - 9ms/step - accuracy: 0.6549 - loss: 0.8493 - val_accuracy: 0.7778 - val_loss: 0.7451
Epoch 10

Resultado Teste
5/5 - 0s - 9ms/step - accuracy: 0.8521 - loss: 0.7331
2/2 - 0s - 21ms/step - accuracy: 0.9722 - loss: 0.5865
